# Trabalho 2 - Dashboard

## Componentes

- Raphael Ramos
- Vitor Gabriel

In [1]:
import sys
sys.path.append('../')

import pandas as pd

data_path = '../data'

df_air_quality = pd.read_csv('{}/india_air_quality_daily_2015_to_2020.csv'.format(data_path))

## Objetivo

Este notebook visa exportar os conjuntos de dados que serão utilizados para os gráficos interativos do Dashboard que será apresentado numa página web, assim como apresentar as suas implementações utilizando o pacote Plotly

## Conjuntos de dados

### AQI anual nas cidades da Índia

Média do AQI anual nas cidades presente no conjunto de dados utilizados, assim como sua classificação categórica (boa, moderada, insalubre e perigosa)

In [19]:
# Função para classificar o AQI
def classify_aqi(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy SG"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Very Unhealthy"
    else:
        return "Hazardous"

df_cities_aqi = df_air_quality[['City', 'Date', 'AQI']].copy()

# Remover linhas com pelo menos uma coluna null
cities_aqi_filtered = df_cities_aqi.dropna().copy()

# Converter a coluna 'Date' para datetime e extrair o ano
cities_aqi_filtered['Date'] = pd.to_datetime(cities_aqi_filtered['Date'])
cities_aqi_filtered['Year'] = cities_aqi_filtered['Date'].dt.year

# Colocar o AQI médio anual de cada cidade
annual_aqi = cities_aqi_filtered.groupby(['City', 'Year'])['AQI'].mean().reset_index()
annual_aqi['AQI_Category'] = annual_aqi['AQI'].apply(lambda x : classify_aqi(x))

# Exportar dataframe para CSV
annual_aqi.to_csv('{}/dashboard_annual_aqi.csv'.format(data_path))

annual_aqi

,City,Year,AQI,AQI_Category
0,Ahmedabad,2015,310.950570,Hazardous
1,Ahmedabad,2016,310.162393,Hazardous
2,Ahmedabad,2017,558.768116,Hazardous
3,Ahmedabad,2018,622.263305,Hazardous
4,Ahmedabad,2019,516.352273,Hazardous
...,...,...,...,...
93,Visakhapatnam,2016,103.976048,Unhealthy SG
94,Visakhapatnam,2017,143.094340,Unhealthy SG
95,Visakhapatnam,2018,122.819018,Unhealthy SG
96,Visakhapatnam,2019,123.442815,Unhealthy SG


### AQI *versus* poluentes PM2.5 e PM10

Média mensal dos poluentes PM10, PM2.5 e do AQI ao no periodo 2015-2020 para atestar a correlação entre eles

In [17]:
df_pollutants_aqi = df_air_quality[['Date', 'PM10', 'PM2.5', 'AQI']]

# Remover linhas com pelo menos uma coluna null
pollutants_aqi_filtered = df_pollutants_aqi.dropna().copy()

# Converter a coluna 'Date' para datetime e extrair o ano e mês
pollutants_aqi_filtered['Date'] = pd.to_datetime(pollutants_aqi_filtered['Date'])
pollutants_aqi_filtered['YearMonth'] = (
    pollutants_aqi_filtered['Date'].dt.year.astype(str) + '-' +
    pollutants_aqi_filtered['Date'].dt.month.astype(str).str.zfill(2)
)

month_aqi_pm10_pm25 = pollutants_aqi_filtered.groupby('YearMonth')[['PM10', 'PM2.5', 'AQI']].mean().reset_index()

# Exportar dataframe para CSV
month_aqi_pm10_pm25.to_csv('{}/dashboard_month_aqi_pm10_pm25.csv'.format(data_path))

month_aqi_pm10_pm25

,YearMonth,PM10,PM2.5,AQI
0,2015-01,276.971290,175.690645,342.290323
1,2015-02,234.939643,153.920357,327.928571
2,2015-03,135.044516,80.338065,256.064516
3,2015-04,187.991333,91.562333,305.266667
4,2015-05,271.050645,97.109355,348.580645
...,...,...,...,...
62,2020-03,87.199142,40.545522,109.666667
63,2020-04,69.094336,30.100687,85.633588
64,2020-05,69.317594,26.813936,86.529903
65,2020-06,56.373368,22.388847,73.094311
